# Dashboard

To view my dashboard, simply run the code cell below! If you would like to run it inline within the Jupyter notebook, comment the last line and un-comment the second-to-last line. You may have to play with `jupyter_height` for it to display fully depending on the size of your screen.

In [136]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

# reading in data and column descriptions
dataset = pd.read_csv("cost-of-living_v2.csv")
dataset.drop("data_quality", axis=1, inplace=True)

desc_df = pd.read_csv("headers.csv", index_col=0, header=None).T
desc_df.drop(["city", "country", "data_quality"], axis=1, inplace=True)

col_list = desc_df.iloc[0].to_numpy()

# renaming columns
x_list = [f"x{i}" for i in range(1, 56)]
col_mapping = {}
for i, og_name in enumerate(x_list):
    col_mapping[og_name] = col_list[i]
dataset = dataset.rename(columns=col_mapping)

app = Dash()

app.layout = html.Div(
    [
        html.H2(children='Cost of Living Dashboard', style={'textAlign':'center'}),
        
    html.Div([
        html.Label("Select Column:"),
        dcc.Dropdown(col_list, col_list[0], id='col-dropdown', style={'width':'50%'}),
        dcc.Graph(id='choropleth', style={'width':'48%', 'display':'inline-block'}),
        dcc.Graph(id='bar-graph', style={'width':'48%', 'display':'inline-block'})
    ], style={"padding": "10px"}),
    html.Div([
        html.Div([
            html.Div([html.Label("X Axis:"),
                  dcc.Dropdown(col_list, col_list[0], id='x-dropdown')], style={'width':'24%', 'display':'inline-block', "padding":"5px"}),
            html.Div([html.Label("Y Axis:"),
                  dcc.Dropdown(col_list, col_list[1], id='y-dropdown')], style={'width':'24%', 'display':'inline-block', "padding":"5px"}),
            html.Div([html.Label("Select Country:"),
                dcc.Dropdown(dataset.country.unique(), 'United States', id='country-dropdown')], style={'width':'24%', "display":"inline-block", "padding":"5px"})
        ], style=dict(display="flex")),
        
        dcc.Graph(id='scatter-plot', style={'width':'48%', 'display':'inline-block'}),
        dcc.Graph(id='pie-chart', style={'width':'48%', 'display':'inline-block'}),
    ], style={"padding": "10px"})
], style={"backgroundColor":"white"})

@app.callback(
    Output('choropleth', 'figure'),
    Input('col-dropdown', 'value'))
def update_map(col_name):
    col_name = col_name if col_name is not None else col_list[0]
    geo = px.data.gapminder().query("year==2007")
    merged = pd.merge(dataset, geo, on=["country"], how="left")

    mean_df = merged.groupby(["country", "iso_alpha", "continent"])[col_list].mean().reset_index()
    mean_not_nan = mean_df[mean_df[col_name].notnull()].reset_index(drop=True)
    mean_not_nan["hover"] = mean_not_nan["country"].astype(str)

    fig = px.scatter_geo(mean_not_nan, locations="iso_alpha", color="continent",
                        hover_name="hover", size=col_name,
                        projection="natural earth", height=600)
    fig.update_layout(title=f"{col_name} Costs Globally")
    return fig

@app.callback(
    Output('bar-graph', 'figure'),
    Input('col-dropdown', 'value'))
def update_bar(col_name):
    col_name = col_name if col_name is not None else col_list[0]
    df = dataset[["city", "country", col_name]]
    top_25 = df.sort_values([col_name], ascending=False).head(25)
    fig = px.bar(top_25, x="city", y=col_name, hover_data=["city", "country", col_name], height=600)
    fig.update_layout(title=f"Top 25 most expensive cities for {col_name}")
    fig.update_layout(xaxis_title="City")
    return fig


@app.callback(
    Output('scatter-plot', 'figure'),
    Input('x-dropdown', 'value'),
    Input('y-dropdown', 'value'))
def update_bar(x_col, y_col):
    fig = px.scatter(dataset, x=x_col, y=y_col, hover_data=["city", "country", x_col, y_col], height=600)
    fig.update_layout(title=f"{y_col} vs {x_col} Costs Globally")
    return fig

@app.callback(
    Output('pie-chart', 'figure'),
    Input('country-dropdown', 'value'))
def update_pie(country):
    mean_df = dataset.groupby(["country"])[col_list].mean().reset_index()

    categories = mean_df.columns.to_numpy()
    costs = mean_df.loc[mean_df["country"]==country, col_list].dropna(axis=1).to_numpy()
    costs = costs.reshape(-1, 1)
    
    df = pd.DataFrame(data=categories[1:], columns=["categories"])
    df["costs"] = costs
    df.loc[df["costs"] < 40, "categories"] = "Other costs"

    fig = px.pie(df, values="costs", names="categories", title=f"Cost of Living Breakdown for {country}", height=600)
    return fig


# app.run(debug=True, jupyter_height=1160, jupyter_width="80%")
app.run(debug=True, jupyter_mode="tab")


Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>